In [1]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
# llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
llm = ChatOpenAI(
    api_key = os.getenv("OPENROUTER_API_KEY"),
    base_url = "https://openrouter.ai/api/v1",
    model = "google/gemini-2.5-flash",
    max_completion_tokens=200

)

In [4]:
from typing import TypedDict, Annotated, List, Optional, Dict, Any, Literal
from langgraph.graph.message import add_messages
from langchain.schema import BaseMessage

class State(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    intent: Optional[Literal["menu", "basic_info", "faq", "ambiguous"]]
    confidence: Optional[float]
    slots: Optional[Dict[str, Any]]

In [26]:
INTENT_PROMPT = """You are an IntentParser for a restaurant assistant.
Your job is to (1) classify the user's message into one or more allowed intents, and (2) extract only the slots defined for each intent.

Allowed intents:
- menu — user asks about items, availability, filtering (veg/non-veg/price/type), or search.
- basic_info — user asks about hours, location, contact, or other venue facts.
- faq — policy-style questions (refunds, delivery areas, allergens policy, tipping, etc.).
- ambiguous — cannot confidently choose (low info or off-domain).

Output format (JSON only):
{
  "tasks": [
    {
      "intent": "menu | basic_info | faq | ambiguous",
      "confidence": 0.0,
      "slots": { /* schema depends on intent; omit unknown/None fields */ }
    }
  ]
}

Extraction policy:
- Return **1–2 tasks**. If the user clearly asks for multiple distinct things (e.g., menu + hours), split into two tasks.
- If you cannot clearly split the asks, return a single task with intent "ambiguous" (confidence < 0.5).

Slot schemas:
- menu
    - search: string | null (free-text query, e.g., "pizza")
    - type: "veg" | "nonveg" | null
    - price_min: number | null (>= 0)
    - price_max: number | null (>= 0)
- basic_info
    - topic: "hours" | "location" | "contact" | "general"
- faq
    - q: string | null (policy-style question in brief)
- ambiguous
    - slots: empty object {}

Rules:
1. Confidence is a real number in [0,1]. If you’re not sure, use intent: "ambiguous" and confidence < 0.5.
2. Include only fields that exist in the schema for the chosen intent. Do not invent keys.
3. Map "veg" synonyms (e.g., "vegetarian", "paneer") to "veg"; map meat words to "nonveg".
4. If the user gives a price range, map to price_min and price_max (leave either as null if missing).
5. No currency symbols; numbers only.
6. Keep outputs concise; JSON only.

Few-shot examples:

User: "do you have veg pizzas under 12?"
{
  "tasks": [
    {
      "intent": "menu",
      "confidence": 0.86,
      "slots": {
        "search": "pizza",
        "type": "veg",
        "price_min": null,
        "price_max": 12
      }
    }
  ]
}

User: "what time do you close on Fridays?"
{
  "tasks": [
    {
      "intent": "basic_info",
      "confidence": 0.9,
      "slots": { "topic": "hours" }
    }
  ]
}

User: "do you deliver to bhaktapur and is there any extra charge?"
{
  "tasks": [
    {
      "intent": "faq",
      "confidence": 0.78,
      "slots": { "q": "delivery area and extra charge" }
    }
  ]
}

User: "veg pizza and your hours?"
{
  "tasks": [
    {
      "intent": "menu",
      "confidence": 0.82,
      "slots": { "search": "pizza", "type": "veg", "price_min": null, "price_max": null }
    },
    {
      "intent": "basic_info",
      "confidence": 0.77,
      "slots": { "topic": "hours" }
    }
  ]
}

User: "hi"
{
  "tasks": [
    {
      "intent": "ambiguous",
      "confidence": 0.2,
      "slots": {}
    }
  ]
}

Final instruction:
Respond with only the JSON object as specified. No markdown, no commentary, no extra text.
"""


In [30]:
import json
import re
from langchain.schema import SystemMessage

def intent_parser(state: State):
    messages = [SystemMessage(content=INTENT_PROMPT)] + state["messages"]
    resp = llm.invoke(messages)

    raw = resp.content.strip()
    # strip ```json ... ``` or ``` ... ```
    if raw.startswith("```"):
        m = re.search(r"```(?:json)?\s*(.*?)\s*```", raw, flags=re.DOTALL|re.IGNORECASE)
        raw = m.group(1).strip() if m else raw

    try:
        parsed = json.loads(raw)
    except json.JSONDecodeError:
        parsed = {"tasks": []}

    tasks = parsed.get("tasks", [])
    if tasks:
        # choose dominant task (highest confidence)
        best = max(tasks, key=lambda t: t.get("confidence", 0) or 0.0)
        intent = best.get("intent", "ambiguous")
        confidence = float(best.get("confidence", 0.0) or 0.0)
        slots = best.get("slots", {}) or {}
    else:
        intent, confidence, slots = "ambiguous", 0.0, {}

    return {
        "messages": [resp],      # append model message
        "tasks": tasks,          # <-- new multi-intent field
        "intent": intent,        # <-- keep single-intent fields for routing today
        "confidence": confidence,
        "slots": slots,
    }


In [31]:
q = "do you guys do anything spicy and also what time do you guys open"
from langchain.schema import HumanMessage
state: State = {"messages": [HumanMessage(content=q)],
                "intent": None, "confidence": None, "slots": {}}
out = intent_parser(state)

In [32]:
out

{'messages': [AIMessage(content='```json\n{\n  "tasks": [\n    {\n      "intent": "menu",\n      "confidence": 0.75,\n      "slots": {\n        "search": "spicy",\n        "type": null,\n        "price_min": null,\n        "price_max": null\n      }\n    },\n    {\n      "intent": "basic_info",\n      "confidence": 0.8,\n      "slots": {\n        "topic": "hours"\n      }\n    }\n  ]\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 980, 'total_tokens': 1106, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'google/gemini-2.5-flash', 'system_fingerprint': None, 'id': 'gen-1760615968-QLTv15bbitIjvyQgPyuH', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--4301d483-66cd-4240-90c1-ddee6de95f4